In [1]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField
from langchain.runnables.hub import HubRunnable

In [3]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini")

In [5]:
prompt1=ChatPromptTemplate.from_template(
    template="What is the city {person} is from?"
)

prompt2=ChatPromptTemplate.from_template(
    template="What Country is the city {city} in? Respond in {language}")

In [6]:
chain1=prompt1|llmGPT|StrOutputParser()

In [9]:
chain2={"city":chain1,"language":itemgetter("language")}|prompt2|llmGPT|StrOutputParser()

In [10]:
print(chain2.invoke(input={"person":"Osama Bin Laden","language":"Bengali"}))

জেদ্দা, একটি শহর যেখানে ওসামা বিন লাদেনের জন্ম হয়েছিল, সৌদি আরবে অবস্থিত।


In [11]:
chain2.get_graph().print_ascii()

           +------------------------------+      
           | Parallel<city,language>Input |      
           +------------------------------+      
                  ***             ***            
                **                   ***         
              **                        **       
+--------------------+                    **     
| ChatPromptTemplate |                     *     
+--------------------+                     *     
           *                               *     
           *                               *     
           *                               *     
    +------------+                         *     
    | ChatOpenAI |                         *     
    +------------+                         *     
           *                               *     
           *                               *     
           *                               *     
  +-----------------+                 +--------+ 
  | StrOutputParser |                 | Lambda | 


<h3>A More Complex Multiple Chain</h3>

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel

llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

prompt1=ChatPromptTemplate.from_template(
    template="generate a {attribute} color. Return the name of the color and nothing else:"
)

prompt2=ChatPromptTemplate.from_template(
    template="what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)

prompt3=ChatPromptTemplate.from_template(
    template="what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)

prompt4=ChatPromptTemplate.from_template(
    template="What is the color of {fruit} and the flag of {country}?"
)

In [25]:
colorChain={"attribute":RunnablePassthrough()} | prompt1 | llmGPT | StrOutputParser()

fruitChain={"color":colorChain}|prompt2|llmGPT|StrOutputParser()

countryChain={"color":colorChain}|prompt3|llmGPT|StrOutputParser()

fruitAndCountryChain=RunnableParallel({"fruit":fruitChain,"country":countryChain})|prompt4|llmGPT|StrOutputParser()

In [26]:
print(fruitAndCountryChain.invoke(input={"attribute":"Yellow"}))

Lemons are typically yellow in color. The flag of Brunei features a field of yellow with a horizontal black stripe and a white stripe. In the center, there is the national emblem of Brunei, which includes a red crescent, a flag, and two waving hands. The overall color scheme includes yellow, black, white, and red.


In [27]:
fruitAndCountryChain=RunnablePassthrough.assign(color=itemgetter('attribute')|prompt1).\
    assign(color=itemgetter('color')|llmGPT).\
    assign(color=itemgetter('color')|StrOutputParser()).\
    assign(fruit=itemgetter('color')|prompt2|llmGPT|StrOutputParser()).\
    assign(country=itemgetter('color')|prompt3|llmGPT|StrOutputParser()).\
    assign(response=prompt4|llmGPT|StrOutputParser())

fruitAndCountryChain.invoke(input={"attribute":"Yellow"})

{'attribute': 'Yellow',
 'color': 'Canary Yellow',
 'fruit': 'Banana',
 'country': 'Colombia',
 'response': 'The color of a banana is typically yellow when ripe, though it can also be green when unripe and brown when overripe. \n\nThe flag of Colombia consists of three horizontal stripes: the top stripe is yellow (which takes up half of the flag), the middle stripe is blue, and the bottom stripe is red.'}

In [28]:
fruitAndCountryChain.get_graph().print_ascii()

              +----------------------+             
              | Parallel<color>Input |             
              +----------------------+             
                  ***            **                
                **                 **              
              **                     **            
       +--------+                      **          
       | Lambda |                       *          
       +--------+                       *          
            *                           *          
            *                           *          
            *                           *          
 +--------------------+          +-------------+   
 | ChatPromptTemplate |          | Passthrough |   
 +--------------------+          +-------------+   
                  ***            **                
                     **        **                  
                       **    **                    
              +-----------------------+            
            